In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import missingno
from tqdm.notebook import tqdm
import os
import matplotlib.pyplot as plt
import seaborn as sns
import math
import sklearn
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor, IsolationForest
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, SGDRegressor, ElasticNetCV, Lasso
from sklearn.metrics import mean_squared_error, SCORERS
from sklearn.kernel_ridge import KernelRidge
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.model_selection import GridSearchCV
from xgboost.sklearn import XGBRegressor
import warnings
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
GD = False
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)


In [ ]:
import random

random.seed(10)
print(random.random())

In [ ]:
raw = pd.read_excel('/kaggle/input/ml-assignment/CustomerDB_assignment_500.xlsx')

In [ ]:
raw.info()

In [ ]:
# Plot graphic of missing values
missingno.matrix(raw, figsize = (30,10))

In [ ]:
to_drop = ['reason']
raw = raw.drop(to_drop, axis=1)

In [ ]:
raw.head()

# Split between test and train sets

In [ ]:
df_train, df_test = sklearn.model_selection.train_test_split(raw, train_size=400, test_size=99)

# ****Remove Outliers ****
(only extreme, depending on the case, we want to avoid reducing the dataset massively as we only have 400 observations)

In [ ]:
sns.boxplot(df_train['incom'])

In [ ]:
Q1 = df_train["incom"].quantile(0.25)

Q3 = df_train["incom"].quantile(0.75)

IQR = Q3 - Q1

Lower_Fence = Q1 - (3 * IQR)

Upper_Fence = Q3 + (3 * IQR)

In [ ]:
max(df_train['incom'])

In [ ]:
# remove EXTREME outliers only
df_train = df_train[~((df_train["incom"] < Lower_Fence) |(df_train["incom"] > Upper_Fence))]

In [ ]:
sns.boxplot(df_train['creddebt'])

In [ ]:
Q1 = df_train["creddebt"].quantile(0.25)

Q3 = df_train["creddebt"].quantile(0.75)

IQR = Q3 - Q1

Lower_Fence = Q1 - (3 * IQR)

Upper_Fence = Q3 + (3 * IQR)

In [ ]:
df_train = df_train[~((df_train["creddebt"] < Lower_Fence) |(df_train["creddebt"] > Upper_Fence))]

In [ ]:
max(df_train['creddebt'])

In [ ]:
#removed extreme outlier
#df_train = df_train[df_train.creddebt < 7]

In [ ]:
sns.boxplot(df_train['hourstv'])

In [ ]:
Q1 = df_train["hourstv"].quantile(0.25)

Q3 = df_train["hourstv"].quantile(0.75)

IQR = Q3 - Q1

Lower_Fence = Q1 - (1.5 * IQR)

Upper_Fence = Q3 + (1.5 * IQR)


In [ ]:
df_train = df_train[~((df_train["hourstv"] < Lower_Fence) |(df_train["hourstv"] > Upper_Fence))]

In [ ]:
sns.boxplot(df_train['address'])

In [ ]:
Q1 = df_train["address"].quantile(0.25)

Q3 = df_train["address"].quantile(0.75)

IQR = Q3 - Q1

Lower_Fence = Q1 - (1.5 * IQR)

Upper_Fence = Q3 + (1.5 * IQR)

In [ ]:
df_train = df_train[~(df_train["address"] > Upper_Fence)]

In [ ]:
sns.boxplot(df_train['employ'])

In [ ]:
raw.employ.value_counts()

In [ ]:
max(df_train['employ'])

In [ ]:
Q1 = df_train["employ"].quantile(0.25)

Q3 = df_train["employ"].quantile(0.75)

IQR = Q3 - Q1

Lower_Fence = Q1 - (3 * IQR)

Upper_Fence = Q3 + (3 * IQR)

In [ ]:
df_train = df_train[~((df_train["employ"] < Lower_Fence) |(df_train["employ"] > Upper_Fence))]

In [ ]:

max(df_train['cardspent'])

In [ ]:
sns.boxplot(df_train['cardspent'])

In [ ]:
Q1 = df_train["cardspent"].quantile(0.25)

Q3 = df_train["cardspent"].quantile(0.75)

IQR = Q3 - Q1

Lower_Fence = Q1 - (3 * IQR)

Upper_Fence = Q3 + (3 * IQR)

In [ ]:
df_train = df_train[~((df_train["cardspent"] < Lower_Fence) |(df_train["cardspent"] > Upper_Fence))]

In [ ]:
#cleaned train set
df_train.describe()

In [ ]:
df_train.info()

# **** split train set between dependent and independent sets****

In [ ]:
df_train_c = df_train.copy()

In [ ]:
X = df_train_c.loc[:, df_train_c.columns != 'incom']
y = df_train_c.loc[:, df_train_c.columns == 'incom']

df_train_ID = df_train.custid
df_test_ID = df_test.custid


X = X.loc[:, X.columns != 'custid']

# ***first glance at variables**

In [ ]:
for i in df_train_c.columns:
    plt.hist(df_train_c[i])
    plt.title(i)
    plt.show()

In [ ]:
#categorical >> coded 0,1
#gender,union,retire,default,marital,homeown,carbuy,polaprty,vote,cardfee,active,churn

#discrete
#jobcat,jobsat,cars,polview,card,cardtype,cardbenefit


In [ ]:
#view only normalized variables that went into the linear regression!
for i in X_train_to_model.columns:
    plt.hist(X_train_to_model[i])
    plt.title(i)
    plt.show()

# Remember
# categorical coded 0,1 >> gender,union,retire,default,marital,homeown,carbuy,polaprty,vote,cardfee,active,churn

# discrete >> jobcat,jobsat,cars,polview,card,cardtype,cardbenefit

# continuous >> employ, debtinc, reside, pets, address,cardtenure, cardspent



# ****Check data skewness

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15,5))

sns.distplot(y, fit=stats.norm, ax=ax[0], kde=False)
stats.probplot(y.incom,  plot=ax[1])
plt.show()
print(f'Fisher-Pearson coeficient of skewness: {stats.skew(y.incom.values):.2f}')

# ****any skewness higher than 0.5 will be transformed

In [ ]:
numerical_columns = X.loc[:, ~X.columns.isin(['gender','union','retire','default','marital','homeown','carbuy','polparty','vote','cardfee','active','churn'])].select_dtypes(include=['int', 'float']).columns
sk = X[numerical_columns].apply(lambda x: stats.skew(x.dropna())).to_frame('Fisher-Pearson Coef')
skw_cols = list(sk[abs(sk['Fisher-Pearson Coef']) > 0.5].index)
sk[abs(sk['Fisher-Pearson Coef']) > 0.5]

# apply logarithm to both X and Y!

In [ ]:
lmbda = 0.0
X[skw_cols] = X[numerical_columns].loc[:, X[numerical_columns].columns.isin(skw_cols)].apply(lambda x: stats.boxcox(1+x, lmbda=lmbda))

In [ ]:
y = y.apply(lambda x: stats.boxcox(1+x, lmbda=lmbda))

In [ ]:
yy = np.exp(y_newpred_2)

In [ ]:
yy.describe()

In [ ]:
# removed skewness, before and after
sk['Fisher-Pearson Coef (After)'] = X[numerical_columns].apply(lambda x: stats.skew(x))
sk[sk.index.isin(skw_cols)]

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15,5))

sns.distplot(y, fit=stats.norm, ax=ax[0], kde=False)
stats.probplot(y.incom,  plot=ax[1])
plt.show()
print(f'Fisher-Pearson coeficient of skewness: {stats.skew(y.incom.values):,.2f}')

# ****selecting features and checking correlation

# numerical cols

In [ ]:
X_disc = X.loc[:,~(X.columns.isin(['jobcat','jobsat','cars','polview','card','cardtype','cardbenefit'])) &
                  (X.columns.isin(numerical_columns))]

X_disc['y'] = y

_, ax = plt.subplots(figsize=(25,15))

sns.heatmap(X_disc.corr(), annot=True, cbar=False, cmap='YlGnBu')
plt.show()

# filter cols that have a correlation with y higher than 0.25

In [ ]:
mask = (abs(X_disc.corr()['y'] >= 0.25))
corr_variables = X_disc.corr()['y'][mask]
corr_variables = list(corr_variables[corr_variables.index != 'y'].index)

corr_variables

# remove independent variables that have high colinearity > 0.75

In [ ]:
_, ax = plt.subplots(figsize=(15,8))

sns.heatmap(X_disc.loc[:, corr_variables].corr(), annot=True, cbar=True, cmap='YlGnBu')
plt.show()

In [ ]:
mask = ((abs(X_disc.loc[:, corr_variables].corr()) > 0.8) & 
        (X_disc.loc[:, corr_variables].corr() != 1.0))
cols = list(X_disc.loc[:, corr_variables].corr()[mask].dropna(how='all', axis=1).columns)

to_remove = []

for i in range(0,len(cols),2):
    to_remove.append(cols[i])
    
continous_features = list(set(corr_variables) - set(to_remove))
continous_features

# let's look at discrete cols

In [ ]:
X_discrete = X.loc[:, X.columns.isin(['jobcat','jobsat','cars','polview','card','cardtype','cardbenefit'])]
X_discrete['y'] = y

_, ax = plt.subplots(figsize=(15,8))

sns.heatmap(X_discrete.corr('spearman'), annot=True, cmap='YlGnBu')
plt.show()

In [ ]:
mask = (((abs(X_discrete.corr('spearman')['y']) >= 0.2) &
       (X_discrete.corr('spearman')['y'] != 1.0)))
X_discrete_cols = list(X_discrete.corr('spearman')['y'][mask].index)
discrete_features = list(set(X_discrete_cols))

In [ ]:
continous_features

In [ ]:
discrete_features

In [ ]:
X_num = X.loc[:, X.columns.isin(continous_features + discrete_features)]
X_num['y'] = y

In [ ]:
sns.pairplot(x_vars=continous_features[:int(len(continous_features)/2)],
             y_vars=['y'],
            data=X_num,
            height=3.5)

sns.pairplot(x_vars=continous_features[int(len(continous_features)/2):],
             y_vars=['y'],
            data=X_num,
            height=3.5)

sns.pairplot(x_vars=discrete_features,
             y_vars=['y'],
            data=X_num,
            height=3.5)

plt.show()

# dichotomous variables (coded 0-1)

In [ ]:
dico = ['gender','union','retire','default','marital','homeown','carbuy','polparty','vote','cardfee','active','churn']

# Training and testing!

In [ ]:
# X, include only the filtered columns
features = continous_features + discrete_features + dico
X = X[features]
y = y


In [ ]:
X_train = X
y_train = y

In [ ]:
#for text, split between x and y
X_test = df_test.loc[:, df_test.columns != 'incom']
y_test = df_test.loc[:, df_test.columns == 'incom']


In [ ]:
X_test.head()

In [ ]:
#let's copy X_train so we can reset and remove variables easily

X_train_to_model = X_train.copy()

In [ ]:
X_train_to_model.head()

# from the whole X_train data set, these are the variables we removed after testing in the linear regression (due to insignificance)

In [ ]:
#removed
#'union','marital','default','polparty','active','vote','carbuy','jobsat','cardtenure','cardfee','gender'
to_drop_from_model = ['union','marital','default','polparty','active','vote','carbuy','jobsat','cardfee','vote','gender']
X_train_to_model = X_train_to_model.drop(to_drop_from_model, axis=1)

# drop the same variables from the test set

In [ ]:
#'union','marital','default','polparty','active','vote','carbuy','jobsat','cardtenure','cardfee'
keep_on_test = ['employ','address','creddebt','cardspent','retire','homeown','churn']
X_test = X_test[keep_on_test]

In [ ]:
X_test.info()

# Linear regression

In [ ]:
linear_reg = sm.OLS(y_train, X_train_to_model)

In [ ]:
results = linear_reg.fit()

In [ ]:
importance = results.coef_[0]

In [ ]:
print(results.summary())

In [ ]:
# standardized coefficients
df_z = X_train_to_model.select_dtypes(include=[np.number]).dropna().apply(stats.zscore)

In [ ]:
linear_reg_stz = sm.OLS(y_train, df_z)

In [ ]:
results_stz = linear_reg_stz.fit()

In [ ]:
print(results_stz.summary())

# Actual vs Predicted

In [ ]:
X_test.head()

# transform the test set,same as for train

In [ ]:
numerical_columns = X_test.loc[:, ~X_test.columns.isin(['gender','union','retire','default','marital','homeown','carbuy','polparty','vote','cardfee','active','churn'])].select_dtypes(include=['int', 'float']).columns
sk = X_test[numerical_columns].apply(lambda x: stats.skew(x.dropna())).to_frame('Fisher-Pearson Coef')
skw_test_cols = list(sk[abs(sk['Fisher-Pearson Coef']) > 0.5].index)
sk[abs(sk['Fisher-Pearson Coef']) > 0.5]

In [ ]:
lmbda = 0.0
X_test[skw_test_cols] = X_test[numerical_columns].loc[:, X_test[numerical_columns].columns.isin(skw_test_cols)].apply(lambda x: stats.boxcox(1+x, lmbda=lmbda))

In [ ]:
# removed skewness, before and after
sk['Fisher-Pearson Coef (After)'] = X_test[numerical_columns].apply(lambda x: stats.skew(x))
sk[sk.index.isin(skw_test_cols)]

In [ ]:
X_test.head()

In [ ]:
y_newpred =  results.predict(X_test) # predict out of sample
print(y_newpred)

In [ ]:
y_newpred = pd.DataFrame(y_newpred)

In [ ]:
y_newpred.describe()

In [ ]:
y_newpred = np.exp(y_newpred)

In [ ]:
y_test['incom']-y_nrepred[0]

# Actual vs Predicted >> Result

In [ ]:
sns.distplot(y_test['incom']-y_newpred[0], axlabel="Test - Prediction")
plt.show()

# Understanding variable importance

we will use scikit to extract coefficients

In [ ]:
n_fold = 5

def rmseModel(m):
    kf = KFold(n_splits=n_fold, random_state=0, shuffle=True).get_n_splits()
    rmse = np.sqrt(-cross_val_score(m, X, y, scoring='neg_mean_squared_error', cv=kf))
    return rmse

In [ ]:
ols_reg = LinearRegression()

In [ ]:
ols_reg_scores = rmseModel(ols_reg)
print(f'OLS Reg RMSE, mean: {np.mean(ols_reg_scores)}, stdv: {np.std(ols_reg_scores)}')

In [ ]:
# fit the model
ols_reg.fit(df_z, y_train)

In [ ]:
# get importance
importance = ols_reg.coef_[0]

In [ ]:
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

In [ ]:
from matplotlib import pyplot

In [ ]:
# Feature Importance
def feature_importance(model, data):
    """
    Function to show which features are most important in the model.
    ::param_model:: Which model to use?
    ::param_data:: What data to use?
    """
    fea_imp = pd.DataFrame({'imp': model.coef_[0], 'col': data.columns})
    fea_imp = fea_imp.sort_values(['imp', 'col'], ascending=[True, False]).iloc[-30:]
    _ = fea_imp.plot(kind='barh', x='col', y='imp', figsize=(20, 10))
    return fea_imp
    

In [ ]:
# Plot the feature importance scores
feature_importance(ols_reg, df_z)

In [ ]:
X_tofold = raw.copy()

In [ ]:
 X_tofold = raw.loc[:,X_tofold.columns != 'incom']

In [ ]:
kf = KFold(5,True, 1)
>>> kf.get_n_splits(X_tofold)
>>> print(kf)

In [ ]:
# enumerate splits
for train_fold, test_fold in kf.split(X_tofold):
	print('train: %s, test: %s' % (train_fold, test_fold))

In [ ]:
KFold(n_splits=2, random_state=None, shuffle=False)
>>> for train_index, test_index in kf.split(X_tolfold):
...     print("TRAIN:", train_index, "TEST:", test_index)
...     X_train_fold, X_test_fold = X[train_index], X[test_index]
...     y_train_fold, y_test_fold = y[train_index], y[test_index]
TRAIN: [2 3] TEST: [0 1]
TRAIN: [0 1] TEST: [2 3]